In [1]:
import pandas as pd
import numpy as np
import os
import numpy as np
import random
from datetime import datetime
import time
import math
import gdown
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import *
from tensorflow.keras.layers import Conv2D, ReLU, MaxPooling2D, Dense, BatchNormalization, GlobalAveragePooling2D, Softmax

try: 
    AUTOTUNE = tf.data.AUTOTUNE
except: 
    AUTOTUNE = tf.data.experimental.AUTOTUNE

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

In [3]:
## Hyper-Parameter 설정
# image resolution
# 최적화 된 값은 아님
RES = 224
# class 수 
N_CLASS = 10
# batch size
N_BATCH = 64
# epoch 수
N_EPOCH = 60
# learning rate
LR = 0.0001

In [4]:
# 데이터 셋 호출 
data_dir = "."
train_dir = os.path.join(data_dir, 'seg_train', 'seg_train')
val_dir = os.path.join(data_dir, 'seg_test', 'seg_test')

train_ds = keras.preprocessing.image_dataset_from_directory(
    train_dir,
    shuffle=True,
    image_size=(RES, RES),
    batch_size=N_BATCH
)
val_ds = keras.preprocessing.image_dataset_from_directory(
    val_dir,
    shuffle=False,
    image_size=(RES, RES),
    batch_size=N_BATCH
)

train_ds = train_ds.prefetch(AUTOTUNE)
val_ds = val_ds.prefetch(AUTOTUNE)

Found 14034 files belonging to 6 classes.
Found 3000 files belonging to 6 classes.


# MoileNet

In [8]:
from tensorflow.keras.applications import MobileNetV3Small
from tensorflow.keras.applications.mobilenet_v3 import preprocess_input
mobilenetv3 = MobileNetV3Small(weights='imagenet', include_top=False, input_shape=(RES, RES, 3))

In [9]:
mobilenetv3.summary()

Model: "MobilenetV3small"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
rescaling (Rescaling)           (None, 224, 224, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv (Conv2D)                   (None, 112, 112, 16) 432         rescaling[0][0]                  
__________________________________________________________________________________________________
Conv/BatchNorm (BatchNormalizat (None, 112, 112, 16) 64          Conv[0][0]                       
___________________________________________________________________________________

In [10]:
def create_model():
    inputs = tf.keras.Input(shape=(RES, RES, 3))

    x = preprocess_input(inputs) 
    x = mobilenetv3(x) # outputd이 7 by 7 이 나옴
    x = GlobalAveragePooling2D()(x) # outputd이 1 by 1 이 나옴
    x = Dense(N_CLASS)(x)
    x = BatchNormalization()(x)
    outputs = Softmax()(x)
    model = keras.Model(inputs=inputs, outputs=outputs) 

    return model

In [11]:
model = create_model()

In [12]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
MobilenetV3small (Functional (None, 1, 1, 1024)        1529968   
_________________________________________________________________
global_average_pooling2d_1 ( (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 10)                10250     
_________________________________________________________________
batch_normalization (BatchNo (None, 10)                40        
_________________________________________________________________
softmax (Softmax)            (None, 10)                0         
Total params: 1,540,258
Trainable params: 1,528,126
Non-trainable params: 12,132
______________________________________________

In [13]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

In [14]:
train_loss = tf.keras.metrics.Mean()
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

test_loss = tf.keras.metrics.Mean()
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

In [15]:
@tf.function 
def train_step(image, label): 
    with tf.GradientTape() as tape: 
        predictions = model(image)
        loss = loss_object(label, predictions) 
        
    gradient = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradient, model.trainable_variables))
    
    train_loss(loss) 
    train_accuracy(label, predictions)

In [16]:
@tf.function 
def test_step(image, label): 
    with tf.GradientTape() as tape: 
        predictions = model(image)
        loss = loss_object(label, predictions) 
        
    # gradient = tape.gradient(loss, model.trainable_variables)
    # optimizer.apply_gradient(zip(gradient, model.trainable_variables))
    
    test_loss(loss) 
    test_accuracy(label, predictions)

In [17]:
for epoch in range(N_EPOCH):
    
    for image, label in train_ds: 
        train_step(image, label)
        
    # for image, label in test_ds: 
    for image, label in val_ds:
        test_step(image, label)
        
    temp = 'epoch : {}, loss : {}, accuracy : {}, test_loss : {}, test_accuracy : {}'
    print(temp.format(epoch+1
                    , train_loss.result() 
                    , train_accuracy.result() * 100 
                    , test_loss.result() 
                    , test_accuracy.result() * 100))
        

epoch : 1, loss : 0.33984464406967163, accuracy : 87.89369201660156, test_loss : 0.2732047736644745, test_accuracy : 90.66667175292969
epoch : 2, loss : 0.27555185556411743, accuracy : 90.27362060546875, test_loss : 0.2598155438899994, test_accuracy : 90.83333587646484
epoch : 3, loss : 0.23846335709095, accuracy : 91.53009796142578, test_loss : 0.2614055871963501, test_accuracy : 90.98888397216797
epoch : 4, loss : 0.21757547557353973, accuracy : 92.30618286132812, test_loss : 0.2641351521015167, test_accuracy : 91.04166412353516
epoch : 5, loss : 0.1999586820602417, accuracy : 92.91862487792969, test_loss : 0.26607927680015564, test_accuracy : 91.0199966430664
epoch : 6, loss : 0.18708981573581696, accuracy : 93.37799072265625, test_loss : 0.26496803760528564, test_accuracy : 91.12777709960938
epoch : 7, loss : 0.17333829402923584, accuracy : 93.86083221435547, test_loss : 0.2639700472354889, test_accuracy : 91.20476531982422
epoch : 8, loss : 0.16143366694450378, accuracy : 94.28530

In [18]:
model.save('classificatio_model_20220221.h5')

C:\Users\Sehwan Yoo\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


## Make map file

### cutmix 알고리즘 (feat. improving 희망 버전) 

In [7]:
def aug(image, label):
    image = tf.image.random_crop(image, [RES, RES, 3])
    image = tf.image.random_flip_left_right(image)
    return image, label

In [11]:
def get_random_map(i, rank):
    # 2번째로 높은 값 출력
    choiced = np.argsort(map.iloc[i, 2:].values, axis=0)[-rank]
    choiced_map = map[map['label'] == choiced].sample(n=1)

    while choiced_map['image_idx'].values == i:
        choiced_map = map[map['label'] == choiced].sample(n=1)
        
    # print(choiced_map['image_idx'])
    # return int(choiced_map['image_idx'].values[0])
    return tf.convert_to_tensor(choiced_map['image_idx'].values[0], dtype=tf.int32)

In [8]:
# 사용안함
def cutmix(images, labels):
    imgs = []; labs = []
    for i in range(N_BATCH):
        APPLY = tf.cast(tf.random.uniform(()) >= 0.5, tf.int32)
        idx = tf.random.uniform((), 0, N_BATCH, tf.int32)

        W = RES
        H = RES
        lam = tf.random.uniform(())
        cut_ratio = tf.math.sqrt(1.-lam)
        cut_w = tf.cast(W * cut_ratio, tf.int32) * APPLY
        cut_h = tf.cast(H * cut_ratio, tf.int32) * APPLY

        cx = tf.random.uniform((), int(W/8), int(7/8*W), tf.int32)
        cy = tf.random.uniform((), int(H/8), int(7/8*H), tf.int32)

        xmin = tf.clip_by_value(cx - cut_w//2, 0, W)
        ymin = tf.clip_by_value(cy - cut_h//2, 0, H)
        xmax = tf.clip_by_value(cx + cut_w//2, 0, W)
        ymax = tf.clip_by_value(cy + cut_h//2, 0, H)

        mid_left = images[i, ymin:ymax, :xmin, :]
        mid_mid = images[idx, ymin:ymax, xmin:xmax, :]
        mid_right = images[i, ymin:ymax, xmax:, :]
        middle = tf.concat([mid_left, mid_mid, mid_right], axis=1)
        top = images[i, :ymin, :, :]
        bottom = images[i, ymax:, :, :]
        new_img = tf.concat([top, middle, bottom], axis=0)
        imgs.append(new_img)

        cut_w_mod = xmax - xmin
        cut_h_mod = ymax - ymin
        alpha = tf.cast((cut_w_mod*cut_h_mod)/(W*H), tf.float32)
        label1 = labels[i]
        label2 = labels[idx]
        
        print(f'label1 : {label1}')
        print(f'label2 : {label2}')
        
        new_label = ((1-alpha)*label1 + alpha*label2)
        
        print(f'new_label : {new_label}')
        
        labs.append(new_label)
        
    new_imgs = tf.reshape(tf.stack(imgs), [-1, RES, RES, 3])
    new_labs = tf.reshape(tf.stack(labs), [-1, N_CLASS])

    return new_imgs, new_labs

In [9]:
train_ds = train_ds.unbatch().map(aug, num_parallel_calls=AUTOTUNE).batch(N_BATCH, drop_remainder=True).map(cutmix, num_parallel_calls=AUTOTUNE).prefetch(AUTOTUNE)

label1 : Tensor("strided_slice_5:0", shape=(), dtype=int32)
label2 : Tensor("strided_slice_6:0", shape=(), dtype=int32)


TypeError: in user code:

    C:\Users\SEHWAN~1\AppData\Local\Temp/ipykernel_6920/3139192020.py:41 cutmix  *
        new_label = ((1-alpha)*label1 + alpha*label2)
    C:\Users\Sehwan Yoo\AppData\Local\Programs\Python\Python39\lib\site-packages\tensorflow\python\ops\math_ops.py:1383 binary_op_wrapper
        raise e
    C:\Users\Sehwan Yoo\AppData\Local\Programs\Python\Python39\lib\site-packages\tensorflow\python\ops\math_ops.py:1367 binary_op_wrapper
        return func(x, y, name=name)
    C:\Users\Sehwan Yoo\AppData\Local\Programs\Python\Python39\lib\site-packages\tensorflow\python\ops\math_ops.py:1710 _mul_dispatch
        return multiply(x, y, name=name)
    C:\Users\Sehwan Yoo\AppData\Local\Programs\Python\Python39\lib\site-packages\tensorflow\python\util\dispatch.py:206 wrapper
        return target(*args, **kwargs)
    C:\Users\Sehwan Yoo\AppData\Local\Programs\Python\Python39\lib\site-packages\tensorflow\python\ops\math_ops.py:530 multiply
        return gen_math_ops.mul(x, y, 